<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [85]:
import requests
import pandas as pd

In [126]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [127]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99,14379,
1,AS YOU LIKE IT,14380,17171,AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r...
2,THE COMEDY OF ERRORS,17172,20372,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
3,THE TRAGEDY OF CORIOLANUS,20373,30346,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...
4,CYMBELINE,30347,30364,CYMBELINE.\r\nLaud we the gods;\r\nAnd let our...


In [128]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

In [129]:
import re

df_toc['text'] = df_toc['text'].str.replace(r'[\n\r\t]', ' ')
df_toc['text'] = df_toc['text'].str.replace('  ', ' ')
df_toc= df_toc.drop(df_toc.index[0])

In [132]:
df_toc['text'] = df_toc['text'].str.replace('  ', ' ')
df_toc['text'].head()

1    AS YOU LIKE IT DRAMATIS PERSONAE. DUKE, living...
2    THE COMEDY OF ERRORS Contents ACT I Scene I. A...
3    THE TRAGEDY OF CORIOLANUS Dramatis Personae CA...
4    CYMBELINE. Laud we the gods; And let our crook...
5    THE TRAGEDY OF HAMLET, PRINCE OF DENMARK Conte...
Name: text, dtype: object

In [133]:
small_data = df_toc['text'][:3].values
data = df_toc['text'].values

In [134]:
len(small_data)

3

In [135]:
len(data)

42

In [136]:
# Encoding

#Gathering All Text
text = " ".join(small_data)

#Character count
chars = list(set(text))

# Lookup tables - mapping chars to nums for numeric representation 
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [137]:
len(chars)

76

In [138]:
# Create the sequence data
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  106178


In [139]:
# Create x & y
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
    y[i, next_char[i]] = 1

In [140]:
x.shape

(106178, 40, 76)

In [68]:
y.shape

(115565, 76)

In [69]:
# build the model: a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [70]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [71]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [72]:
# fit the model

model.fit(x, y,
          batch_size=32,
          epochs=20,
          callbacks=[print_callback])

Train on 115565 samples
Epoch 1/20
115552/115565 [============================>.] - ETA: 0s - loss: 2.4669
----- Generating text after Epoch: 0
----- Generating with seed: "mio, command him away.    DROMIO OF EPHE"
mio, command him away.    DROMIO OF EPHEUS. Mirovat     be.   MCLOON. With     De tblln.    Da’s suce wittue an fibllts.    Whorsen souk.  I CIMAIUS. ORANS. PMALEM. I sace mat is smass !    CECENI. Neler, the      poule lave therre ta douf mirmis.    HeRLDHN. Pawllound grwan cote anest ther                     No to deey udverst  To Ipae?    MEamr qurun;      Co     CESEN.NPAINN SGARIMN. SOosr'la’os yin rfan ould an       Woro feed     
115565/115565 [==============================] - 63s 546us/sample - loss: 2.4669
Epoch 2/20
115488/115565 [============================>.] - ETA: 0s - loss: 2.0416- ETA
----- Generating text after Epoch: 1
----- Generating with seed: "men,      The common file- a plague! tri"
men,      The common file- a plague! tringen and,  The d'des serad 

In [111]:
## okay, small data done, time for the big boy

data.shape

(42,)

In [141]:
# Encoding

#Gathering All Text
text = " ".join(data)

#Character count
chars = list(set(text))

# Lookup tables - mapping chars to nums for numeric representation 
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [142]:
len(chars)

103

In [143]:
# Create the sequence data
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  2808472


In [144]:
# Create x & y
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
    y[i, next_char[i]] = 1

In [145]:
x.shape

(2808472, 40, 103)

In [146]:
y.shape

(2808472, 103)

In [147]:
# build the model: a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [148]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [149]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [151]:
# fit the model

model.fit(x, y,
          batch_size=32,
          epochs=10,
          callbacks=[print_callback])

Train on 2808472 samples
Epoch 1/10
2808448/2808472 [============================>.] - ETA: 0s - loss: 1.7560
----- Generating text after Epoch: 0
----- Generating with seed: "es doom men to death.    [Rises] But whe"
es doom men to death.    [Rises] But when with prarishdes a crax of this; Where sweet sit precilliillace, whoming fones frignd. [A jottus? The cures fire Thon I guen, strangeres friends. I shall are the parences or give the sidddy sands But white. Your say, I know with thee suns report of enemanteds. Somedero. SECOND GEWSTEM. He came natus, a faith? DUCTOR. By myself Speak our love Pattoudia, SERVANTIA. S Attenousus: To end the’s like s
2808472/2808472 [==============================] - 1211s 431us/sample - loss: 1.7560
Epoch 2/10
2808384/2808472 [============================>.] - ETA: 0s - loss: 1.5877
----- Generating text after Epoch: 1
----- Generating with seed: "oo little for your thief, your true man "
oo little for your thief, your true man wisely Addess? No, sir,

C:\Users\timro\anaconda3\envs\U4-S2-Neural-Networks\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


ity, to her field yet change, Unto husband thy time againer you shidps daskety. COSSIOM. If you, heirlic, he sin; So embrave and man, is not me upon, advised! To me. Nay, under our nailot in heren? CATESBY. No wills wintectly eyes wife. Well, there wast new haste The Sounder that thou like thee They need—that devil gone! Repess great themselves’s othal
2808472/2808472 [==============================] - 1233s 439us/sample - loss: 1.4867
Epoch 6/10
2808352/2808472 [============================>.] - ETA: 0s - loss: 1.4724
----- Generating text after Epoch: 5
----- Generating with seed: "ll. To-morrow, sir, I wrestle for my cre"
ll. To-morrow, sir, I wrestle for my creech of died to deign That I would meet evoling behall’d but Agan? SToken at very sweet? I’ll now suint, Alack stood the kingdom But being cure Ratcin as so's child. A mark love it, what an one of and eye, Look my lives, and this lives me secuse, There's a mark honest more than talk, and curtain find emolleg? Hang itsy is, if 

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN